## PiecewiseJerkSpeed 
该Task的相关内容：modules/planning/tasks/optimizers/piecewise_jerk_speed/piecewise_jerk_speed_nonlinear_optimizer

根据Path与障碍物信息，生成平滑的速度曲线t-s/v/a

在前面的几个decider中，根据障碍物与道路信息生成t-sbound对自车状态进行约束

施工中,目前目前已知的问题：
penalty_v 速度惩罚系数


### qp的一般形式
$$
minimize \frac{1}{2} \cdot x^T \cdot P \cdot x + q \cdot x 
\\
s.t. LB \leq A\cdot x \leq UB
$$

### Cost
整理后的cost：
$$
f(s(t)) = w_a\cdot \sum_{i=0}^{n-1} a^2 + w_j\cdot\sum_{i=0}^{n-1} j^2 \\
+w_{refs}\cdot \sum_{i=0}^{n-1}(s_i - sref_i)^2 + w_{refv}\cdot\sum_{i=0}^{n-1}  (v_i - vref_i)^2\\
+w_{ends}\cdot (s_{n-1} - s_{end})^2 + w_{endv}\cdot (v_{n-1} - v_{end})^2 + w_{enda}\cdot (a_{n-1} - a_{end})^2 \\
$$

其中：

第一行代表舒适性，降低加速度a和加加速度jerk

第二行代表安全性，尽可能在指定的时间到达指定的位置，同时尽可能保证匀速行驶，减少加减速频率

第三行表示终点状态的权重

### 将cost转为qp的格式 - P和q矩阵
为了方便以6个点为例，这部分的源码在：modules/planning/math/piecewise_jerk/piecewise_jerk_speed_problem

#### P矩阵：
sf ~ scale_factor 各阶的放大系数

pv ~ penalty_v 速度惩罚系数

$$\begin{vmatrix}
\frac{w_s}{sf_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&\frac{w_s}{sf_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&\frac{w_s}{sf_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&\frac{w_s}{sf_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&\frac{w_s}{sf_0^2}&0&0&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&\frac{w_s+w_{ends}}{sf_0^2}&0&0&0&0&0&0&0&0&0&0&0&0\\
\\
0&0&0&0&0&0&\frac{w_v+pv_0}{sf_1^2}&0&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&\frac{w_v+pv_1}{sf_1^2}&0&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&\frac{w_v+pv_2}{sf_1^2}&0&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&\frac{w_v+pv_3}{sf_1^2}&0&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&\frac{w_v+pv_4}{sf_1^2}&0&0&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&\frac{w_v+pv_5+w_{endv}}{sf_1^2}&0&0&0&0&0&0\\
\\
0&0&0&0&0&0&0&0&0&0&0&0&\frac{w_a+\frac{2\cdot w_j}{\Delta s^2}}{sf_2^2}&0&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\cdot w_j}{\Delta s^2}&\frac{w_a+\frac{2\cdot w_j}{\Delta s^2}}{sf_2^2}&0&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\cdot w_j}{\Delta s^2}&\frac{w_a+\frac{2\cdot w_j}{\Delta s^2}}{sf_2^2}&0&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\cdot w_j}{\Delta s^2}&\frac{w_a+\frac{2\cdot w_j}{\Delta s^2}}{sf_2^2}&0&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\cdot w_j}{\Delta s^2}&\frac{w_a+\frac{2\cdot w_j}{\Delta s^2}}{sf_2^2}&0\\
0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&0&\frac{-2\cdot w_j}{\Delta s^2}&\frac{w_a+\frac{2\cdot w_j}{\Delta s^2}+w_{enda}}{sf_2^2}\\
\end{vmatrix}$$

#### q矩阵：

$$\begin{vmatrix}
\frac{-2w_s\cdot s_{ref0}}{sf_0}\\
\frac{-2w_s\cdot s_{ref1}}{sf_0}\\
\frac{-2w_s\cdot s_{ref2}}{sf_0}\\
\frac{-2w_s\cdot s_{ref3}}{sf_0}\\
\frac{-2w_s\cdot s_{ref4}}{sf_0}\\
\frac{-2w_s\cdot s_{ref5 }-2w_{ends}\cdot s_{end}}{sf_0}\\
\frac{-2w_v\cdot v_{ref0}}{sf_1}\\
\frac{-2w_v\cdot v_{ref1}}{sf_1}\\
\frac{-2w_v\cdot v_{ref2}}{sf_1}\\
\frac{-2w_v\cdot v_{ref3}}{sf_1}\\
\frac{-2w_v\cdot v_{ref4}}{sf_1}\\
\frac{-2w_v\cdot v_{ref5 }-2w_{endv}\cdot v_{end}}{sf_1}\\
0\\
0\\
0\\
0\\
0\\
\frac{-2w_a\cdot a_{end}}{sf_2}\\
\end{vmatrix}$$


#### Bound
与PiecewiseJerkPath基本一致
其中：
sbound来自于decider中通过DP进行求解
v_bound为（0 ，max(当前车速，最大允许车速）） - 应该是防止高速进入自动驾驶模式时猛刹
a_bound默认为（-6 ， 2）
jerk_bound默认为（-4 ， 2）